In [ ]:
import os
import datetime

#打印时间
def printbar():
    nowtime = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    print("\n"+"=========="*8 + "%s"%nowtime)

#mac系统上pytorch和matplotlib在jupyter中同时跑需要更改环境变量
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE" 

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt 
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset

#样本数量
n = 400

# 生成测试用数据集
X = 10 * torch.rand((n, 2)) - 5.0
w0 = torch.tensor([[2.0], [-3.0]])
b0 = torch.tensor([[10.0]])
Y = X@w0 + b0 + torch.normal(0.0, 2.0, size=(n, 1))

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

plt.figure(figsize=(12, 5))
ax1 = plt.subplot(121)
ax1.scatter(X[:, 0].numpy(), Y[:, 0].numpy(), c="b", label="samples")
ax1.legend()
plt.xlabel("x1")
plt.ylabel("y", rotation=0)

ax2 = plt.subplot(122)
ax2.scatter(X[:, 1].numpy(), Y[:, 0].numpy(), c="g", label="samples")
ax2.legend()
plt.xlabel("x2")
plt.ylabel("y", rotation=0)

plt.show()

In [ ]:
ds = TensorDataset(X, Y)
ds_train, ds_valid = torch.utils.data.random_split(ds, [int(n * 0.7), n - int(n * 0.7)])
dl_train = DataLoader(ds_train, batch_size=10, shuffle=True)
dl_valid = DataLoader(ds_valid, batch_size=10)

In [ ]:
import torchkeras
from torchkeras import Model

class LinearRegression(Model):
    def __init__(self):
        super(LinearRegression, self).__init__()
        self.fc = nn.Linear(2, 1)
    
    def forward(self, x):
        return self.fc(x)

model = LinearRegression()

In [ ]:
model.summary(input_shape=(2,))

In [ ]:
def mean_absolute_error(y_pred, y_true):
    return torch.mean(torch.abs(y_pred - y_true))

def mean_absolute_percent_error(y_pred, y_true):
    epsilon = 1e-7
    return torch.mean((torch.abs(y_pred - y_true) + epsilon) / (torch.abs(y_true) + epsilon))

model.compile(loss_func = nn.MSELoss(),
              optimizer= torch.optim.Adam(model.parameters(), lr=0.01),
              metrics_dict={
                  "mae": mean_absolute_error,
                  "mape": mean_absolute_percent_error
                  })

dfhistory = model.fit(200, dl_train=dl_train, dl_val=dl_valid, log_step_freq=20)

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

w = model.state_dict()["fc.weight"]
b = model.state_dict()["fc.bias"]

fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(12, 5))
ax1.scatter(X[:, 0], Y[:, 0], c="b", label="samples")
ax1.plot(X[:, 0], w[0, 0] * X[:, 0] + b[0], "-r", linewidth=5.0, label="model")
ax1.legend()
plt.xlabel("x1")
plt.ylabel("y", rotation = 0)

ax2.scatter(X[:, 1], Y[:, 0], c="g", label="samples")
ax2.plot(X[:, 1], w[0, 1] * X[:, 1] + b[0], "-r", linewidth=5.0, label="model")
ax2.legend()
plt.xlabel("x2")
plt.ylabel("y", rotation = 0)

plt.show()


In [ ]:
dfhistory.tail()

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import matplotlib.pyplot as plt

def plot_metric(dfhistory, metric):
    train_metrics = dfhistory[metric]
    val_metrics = dfhistory['val_' + metric]
    epochs = range(1, len(train_metrics) + 1)
    plt.plot(epochs, train_metrics, 'bo--')
    plt.plot(epochs, val_metrics, 'ro-')

    plt.title('Training and validation '+ metric)
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.legend(['train_' + metric, 'val_' + metric])
    plt.show()

In [ ]:
plot_metric(dfhistory, "loss")

In [ ]:
plot_metric(dfhistory, "mae")

In [ ]:
plot_metric(dfhistory, "mape")

In [ ]:
model.evaluate(dl_valid)

In [ ]:
dl = DataLoader(TensorDataset(X))
model.predict(dl)[0:10]

In [ ]:
# DNN
import numpy as np 
import pandas as pd 
from matplotlib import pyplot as plt
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset

#正负样本数量
n_positive, n_negative = 1000, 1000

#生成正样本, 小圆环分布
r_p = torch.normal(5.0, 1.0, size=[n_positive, 1])  # random r
theta_p = 2 * np.pi * torch.rand([n_positive, 1])   # random theta
Xp = torch.cat([r_p * torch.cos(theta_p), r_p * torch.sin(theta_p)], axis=1)
Yp = torch.ones_like(r_p)

#生成负样本, 大圆环分布
r_n = torch.normal(8.0, 1.0, size=[n_negative, 1]) 
theta_n = 2 * np.pi * torch.rand([n_negative, 1])
Xn = torch.cat([r_n * torch.cos(theta_n), r_n * torch.sin(theta_n)], axis=1)
Yn = torch.zeros_like(r_n)

#汇总样本
X = torch.cat([Xp, Xn], axis=0)
Y = torch.cat([Yp, Yn], axis=0)

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

plt.figure(figsize=(6, 6))
axe = plt.subplot(111)
axe.scatter(Xp[:, 0].numpy(), Xp[:, 1].numpy(), c='r', label='positive')
axe.scatter(Xn[:, 0].numpy(), Xn[:, 1].numpy(), c='g', label='negative')
axe.legend()
plt.show()

In [ ]:
ds = TensorDataset(X, Y)
nb_dl_train = int((n_positive + n_negative) * 0.7)
nb_dl_valid = (n_positive + n_negative) - nb_dl_train
ds_train, ds_valid = torch.utils.data.random_split(ds, [nb_dl_train, nb_dl_valid])
dl_train = DataLoader(ds_train, batch_size=10, shuffle=True)
dl_valid = DataLoader(ds_valid, batch_size=10)

In [ ]:
from torchkeras import Model

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 4)
        self.activation1 = nn.ReLU()
        self.fc2 = nn.Linear(4, 8)
        self.activation2 = nn.ReLU()
        self.fc3 = nn.Linear(8, 1)
        self.activation3 = nn.Sigmoid()

    # 正向传播
    def forward(self, x):
        x = self.fc1(x)
        x = self.activation1(x)
        x = self.fc2(x)
        x = self.activation2(x)
        x = self.fc3(x)
        x = self.activation3(x)
        return x

model = Model(Net())
model.summary(input_shape =(2,))

In [ ]:
def accuracy(y_pred, y_true):
    y_pred = torch.where(y_pred > 0.5, \
        torch.ones_like(y_pred, dtype=torch.float32), torch.zeros_like(y_pred, dtype=torch.float32))
    acc = torch.mean(1 - torch.abs(y_true - y_pred))
    return acc

model.compile(loss_func=nn.BCELoss(), optimizer=torch.optim.Adam(model.parameters(), lr=0.01), \
    metrics_dict={"accuracy": accuracy})

dfhistory = model.fit(100, dl_train=dl_train, dl_val=dl_valid, log_step_freq=10)

In [ ]:
dfhistory.tail()

In [ ]:
# 结果可视化
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(12, 5))

ax1.scatter(Xp[:, 0], Xp[:, 1], c="r", label="positive")
ax1.scatter(Xn[:, 0], Xn[:, 1], c="g", label="negative")
ax1.legend()
ax1.set_title("y_true")

Xp_pred = X[torch.squeeze(model.forward(X) >= 0.5)]
Xn_pred = X[torch.squeeze(model.forward(X) < 0.5)]

ax2.scatter(Xp_pred[:, 0], Xp_pred[:, 1], c="r", label="positive")
ax2.scatter(Xn_pred[:, 0], Xn_pred[:, 1], c="g", label="negative")
ax2.legend()
ax2.set_title("y_pred")

plt.show()

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import matplotlib.pyplot as plt

def plot_metric(dfhistory, metric):
    train_metrics = dfhistory[metric]
    val_metrics = dfhistory['val_' + metric]
    epochs = range(1, len(train_metrics) + 1)
    plt.plot(epochs, train_metrics, 'bo--')
    plt.plot(epochs, val_metrics, 'ro-')
    plt.title('Training and validation '+ metric)
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.legend(['train_' + metric, 'val_' + metric])
    plt.show()

In [ ]:
plot_metric(dfhistory, 'loss')

In [ ]:
plot_metric(dfhistory, 'accuracy')

In [ ]:
model.evaluate(dl_valid)

In [ ]:
model.predict(dl_valid)[0:10]